In [1]:
from __future__ import print_function

In [3]:
#Toy dataset
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
#Column labels.
header = ["color", "diameter", "label"]

In [7]:
#回傳一集合, 去掉重複的元素
def unique_vals(rows, col):
    return set([row[col] for row in rows])

#unique_vals(training_data, 2)

In [15]:
#宣告一個字典counts, label欄位計數有重複加一,回傳此字典
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

#class_counts(training_data)


{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [18]:
#回傳是否int,float數值變數
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

#is_numeric('string')

False

In [27]:
# Question(欄位, 數值)
# Question實現match會將參數example[該欄位]如果是數值欄位比較是否>=參數數值, 字串則比較是否相等
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        #從header中取出對應欄位的字串來顯示
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

#q = Question(0, 'Green')
#print(q)
#print(q.match(training_data[0]))
#print(q.match(training_data[1]))
#q2=Question(1, 3)

Is color == Green?
True
False


In [31]:
#對每筆資料檢查分割條件, 如果question符合,則分進true_rows集合
#回傳true_rows,false_rows兩個子節點
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

#node_true_rows, node_false_rows = partition(training_data, Question(0, 'Red'))
#print(node_true_rows)
#print(node_false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [68]:
#計算吉尼係數, rows是一組物件計算其純度
def gini(rows):
    #計數每種標籤下的數量, 得到字典counts
    counts = class_counts(rows)
    #設不純度=1
    impurity = 1
    for lbl in counts:
        print(lbl)
        #計算p
        prob_of_lbl = counts[lbl] / float(len(rows))
        #計算1-sum(pi**2)
        impurity -= prob_of_lbl**2
    return impurity

#lots_of_mixing = [['Apple'],
#                  ['Orange'],
#                  ['Grape'],
#                  ['Grapefruit'],
#                  ['Blueberry']]
#gini(lots_of_mixing)

#計算gain, 
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# 先得到分割前的純度current_uncertainty
current_uncertainty = gini(training_data)
counts = class_counts(training_data)
print(counts)
#print(current_uncertainty)
# 設一個分割條件A, 分成兩份
#true_rows, false_rows = partition(training_data, Question(0, 'Green'))
#算出gain
#gain1 = info_gain(true_rows, false_rows, current_uncertainty)
#print(gain1)
#print(true_rows)
#print(false_rows)

#true_rows, false_rows = partition(training_data, Question(0,'Red'))
#gain2 = info_gain(true_rows, false_rows, current_uncertainty)
#print(gain2)
#print(true_rows)
#print(false_rows)

Apple
Grape
Lemon
{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [49]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    #當前節點狀態算出不純度
    current_uncertainty = gini(rows)
    #得到feature個數, 最後一個是label
    n_features = len(rows[0]) - 1  # number of columns
    #遍歷feature
    for col in range(n_features):  # for each feature
        #得到該feature下的所有數值集合
        values = set([row[col] for row in rows])  # unique values in the column
        #遍歷該集合
        for val in values:  # for each value
            #該集合每個取出當成分割條件
            question = Question(col, val)
            #嘗試分割
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            #如果沒有分割成功則跳過
            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            #計算gain
            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


best_gain, best_question = find_best_split(training_data)
best_question

Is diameter >= 3?

In [52]:
#建立樹葉class, predictions表示分類結果
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

#建立樹node, 包含分割條件跟左右子分支
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [53]:
#遞迴建立樹
# 1)找出最佳分割條件
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

#印出樹
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [55]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [58]:
#由根節點開始走, 檢查每個節點分割條件, 決定往左或右分支前進, 遇到樹葉則輸出結果
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [57]:
classify(training_data[0], my_tree)

{'Apple': 1}

In [59]:
#樹葉結果用機率表示
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [60]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [61]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [62]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [63]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
